# Data preparation for the learning phase

Install of the library which contains all data

In [ ]:
pip install fastf1

In [37]:
import fastf1
import pandas as pd
import csv

Getting all data from most recent years with same compounds tyres types, infact from 2019 avaiable compounds are SOFT, MEDIUM, HARD, INTERMEDIATE, WET

In [ ]:
schedules = []
for year in [2019, 2020, 2021, 2022, 2023]:
    schedules.append(fastf1.get_event_schedule(year))

## Extraction

Preparing list of colums which have to be selected in the tables of interest:
- Laps table: Driver, DriverNumber, LapNumber, Compound, TyreLife
- Results table: DriverNumber, Position, GridPosition
- Weather Data table: AirTemp, Humidity, Pressure, Rainfall, TrackTemp, WindSpeed

In [ ]:
laps_list = [False, True, True, False, True, False,
       False, False, False, False, False,
       False, False, False,
       False, False, False, False, False,
       True, True, False, False, False,
       False, False, False, False, False,
       False, False]

results_list = [True, False, False, False, False,
       False, False, False, False, False,
       False, False, True, False,
       True, False, False, False, False, False, False]

weather_list = [False, True, True, True, True, True,
       False, True]

Loading of all the data and extracting only column of interest.

About race laps is extracted for each driver the first lap

About weather conditions is selected only information about the first lap

If the code generate an error it is normal because of reached maximum number of request per hour (200/h)

In [ ]:
peryear_races = []
peryear_races_results = []
peryear_races_laps = []
peryear_races_weather = []

i = 0

for schedule in schedules:
    peryear_races.append({})
    peryear_races_results.append({})
    peryear_races_laps.append({})
    peryear_races_weather.append({})
    
    for round in range(1, len(schedule)-1):
        # loading data for each race per year
        peryear_races[i][round] = schedule.get_event_by_round(round).get_race()
        peryear_races[i][round].load()
        
        # extracting only columns of interests
        peryear_races_results[i][round] = peryear_races[i][round].results.loc[:, results_list]

        peryear_races_laps[i][round] = peryear_races[i][round].laps.loc[:, laps_list].pick_lap(1.0)

        peryear_races_weather[i][round] = peryear_races[i][round].weather_data.loc[:, weather_list].head(1)

        # Preparing values in order to be used to train the model:
        # - if Rainfall is False (no rain) -> 0
        # - if Rainfall is True (rain) -> 1
        if peryear_races_weather[i][round]['Rainfall'].bool() == False:
            peryear_races_weather[i][round]['Rainfall'] = 0
        else: 
            peryear_races_weather[i][round]['Rainfall'] = 1
    
    i+=1
    

Merge results and first lap information and adding of round name and year for each race

Adding for each race weather condition information about round name and year of attendance

For both kind of information is created a directory called './data', where csv files are saved

In [ ]:
merge_lapsresults = []
base_year = 2019
i = 0

# Preparing columns of csv files will store races and weather information
results_header= pd.DataFrame(columns=['Compound','TyreLife','Position','GridPosition','Round','Year'])
results_header.to_csv('./data/results.csv')

weather_header=pd.DataFrame(columns=['AirTemp', 'Humidity', 'Pressure', 'Rainfall', 'TrackTemp', 'WindSpeed', 'Round', 'Year'])
weather_header.to_csv('./data/weather.csv')


for schedule in schedules: 
    merge_lapsresults.append({})
      
    for round in range(1, len(schedule)-1):
        year = base_year+i

        # join of peryear_races_laps and peryear_races_results on 'DriverNumber' column
        merge_lapsresults[i][round] = pd.merge(peryear_races_laps[i][round], peryear_races_results[i][round], on=["DriverNumber"])
        # adding round and year information
        merge_lapsresults[i][round]['Round'] = round
        merge_lapsresults[i][round]['Year'] = year

        merge_lapsresults[i][round] = merge_lapsresults[i][round].loc[:, [False,False,False,True,True,False,True,True,True,True]]
                            # Driver, DriverNumber, LapNumber, Compound, TyreLife, FreshTyre, Team, Position, GridPosition, Round, Year
                            # Compound, TyreLife, Position, GridPosition, Round, Year

        # Changing values saved for Compound from string to integer, in order to be ready to train the model:
        # - SOFT -> 0
        # - MEDIUM -> 1
        # - HARD -> 2
        # - INTERMEDIATE -> 3
        # - WET -> 4
        for driver in list(merge_lapsresults[i][round]['Compound'].index):
            if merge_lapsresults[i][round]['Compound'][driver] == 'SOFT':
                merge_lapsresults[i][round]['Compound'][driver] = 0
            elif merge_lapsresults[i][round]['Compound'][driver] == 'MEDIUM':
                merge_lapsresults[i][round]['Compound'][driver] = 1
            elif merge_lapsresults[i][round]['Compound'][driver] == 'HARD':
                merge_lapsresults[i][round]['Compound'][driver] = 2
            elif merge_lapsresults[i][round]['Compound'][driver] == 'INTERMEDIATE':
                merge_lapsresults[i][round]['Compound'][driver] = 3
            elif merge_lapsresults[i][round]['Compound'][driver] == 'WET':
                merge_lapsresults[i][round]['Compound'][driver] = 4

        # adding round and year information to weather dataframe
        peryear_races_weather[i][round]['Round'] = round
        peryear_races_weather[i][round]['Year'] = year
        
        # Saving dataframes in csv files
        merge_lapsresults[i][round].to_csv('./data/results.csv', mode='a', header=False)
        peryear_races_weather[i][round].to_csv('./data/weather.csv', mode='a', header=False)

    i+=1

# Merge of the two tables

Merge between drivers information and weather information, then saving on csv file

In [ ]:
merge = []

i = 0

merge_header = pd.DataFrame(columns=['Compound', 'TyreLife', 'Position', 'GridPosition',
       'Round', 'Year', 'AirTemp', 'Humidity', 'Pressure', 'Rainfall',
       'TrackTemp', 'WindSpeed'])

merge_header.to_csv('./data/merged_results.csv')

for schedule in schedules:
    merge.append({})

    for round in range(1, len(schedule)-1):
        merge[i][round] = pd.merge(merge_lapsresults[i][round], peryear_races_weather[i][round], on=['Round', 'Year'])
        
        merge[i][round].to_csv('./data/merged_results.csv', mode='a', header=False)
        
    i+=1

# Races name, year and round number

Creating a pandas dataframe to save the link between year, round number and name of the round. Then saved in a csv file

In [34]:
gran_prix = {}
base_year = 2019
i = 0

for schedule in schedules: 
    # gran_prix.append({})
    year = base_year+i
    gran_prix[year] = {}
    for round in range(1, len(schedule)-1):
        

        round_name = str(peryear_races[i][round]).split(': ')[1].split(' - ')[0]

        gran_prix[year][round] = round_name
    
    i+=1



In [64]:
gran_prix_pd = pd.DataFrame(gran_prix)
gran_prix_pd.to_csv('./data/gran_prix.csv')